# Kubernetes Cluster Demo Notebook

This notebook demonstrates the capabilities of our three-node Kubernetes cluster:
- **Tower**: Control plane + PostgreSQL + pgAdmin
- **AGX**: Worker node (GPU-enabled)
- **Nano**: Worker node (GPU-enabled, resource-constrained)

## Features Demonstrated
1. 🐘 PostgreSQL with pgvector extension
2. 🎯 Vector similarity search
3. 📊 Cluster resource monitoring
4. 🔗 Service connectivity testing
5. 🤖 GPU resource detection
6. 📈 Data visualization

## 1. Environment Setup and Imports

In [ ]:
# Core libraries
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Database connectivity
import psycopg2
from psycopg2.extras import RealDictCursor

# Kubernetes client
try:
    from kubernetes import client, config
    k8s_available = True
except ImportError:
    print("Installing Kubernetes client...")
    !pip install kubernetes
    from kubernetes import client, config
    k8s_available = True

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 All libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🕐 Notebook started at: {datetime.now()}")

## 2. Cluster Configuration and Connectivity

In [ ]:
# Database configuration
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'postgres-service'),
    'port': os.getenv('POSTGRES_PORT', '5432'),
    'database': os.getenv('POSTGRES_DB', 'vectordb'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', 'myscretpassword')
}

# Service endpoints
SERVICES = {
    'postgres': f"{DB_CONFIG['host']}:{DB_CONFIG['port']}",
    'pgadmin': 'pgadmin-service:80',
    'fastapi_nano': 'fastapi-nano-service:8000'  # Runs exclusively on nano node
}

print("🔧 Configuration loaded:")
print(f"  📊 Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}")
print(f"  🎛️  pgAdmin: {SERVICES['pgadmin']}")
print(f"  🚀 FastAPI: {SERVICES['fastapi_nano']} (nano node only)")
print(f"  ⚠️  Note: FastAPI requires nano node to be Ready")

## 3. Database Connection and pgvector Setup

In [ ]:
def get_db_connection():
    """Establish database connection with error handling"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return None

# Test database connection
print("🔌 Testing database connection...")
conn = get_db_connection()

if conn:
    with conn.cursor() as cur:
        # Check pgvector extension
        cur.execute("SELECT extname, extversion FROM pg_extension WHERE extname = 'vector';")
        result = cur.fetchone()
        
        if result:
            print(f"✅ pgvector extension: v{result[1]}")
        else:
            print("⚠️  pgvector extension not found, installing...")
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
            conn.commit()
            print("✅ pgvector extension installed")
        
        # Get database info
        cur.execute("SELECT version();")
        version = cur.fetchone()[0]
        print(f"🐘 PostgreSQL: {version.split(',')[0]}")
        
    conn.close()
    print("✅ Database connection successful!")
else:
    print("❌ Could not connect to database")

## 4. Kubernetes Cluster Monitoring

In [ ]:
def get_cluster_info():
    """Get comprehensive cluster information"""
    try:
        # Load kube config
        config.load_incluster_config()  # For in-cluster access
    except:
        try:
            config.load_kube_config()  # For local access
        except:
            print("⚠️  Could not load Kubernetes config")
            return None
    
    v1 = client.CoreV1Api()
    apps_v1 = client.AppsV1Api()
    
    cluster_info = {
        'nodes': [],
        'pods': [],
        'services': [],
        'deployments': []
    }
    
    try:
        # Get nodes
        nodes = v1.list_node()
        for node in nodes.items:
            node_info = {
                'name': node.metadata.name,
                'status': 'Ready' if any(condition.type == 'Ready' and condition.status == 'True' 
                                       for condition in node.status.conditions) else 'NotReady',
                'arch': node.status.node_info.architecture,
                'os': node.status.node_info.operating_system,
                'kernel': node.status.node_info.kernel_version,
                'cpu': node.status.allocatable.get('cpu', 'unknown'),
                'memory': node.status.allocatable.get('memory', 'unknown'),
                'gpu': node.status.allocatable.get('nvidia.com/gpu', '0')
            }
            cluster_info['nodes'].append(node_info)
        
        # Get pods
        pods = v1.list_pod_for_all_namespaces()
        for pod in pods.items:
            if pod.metadata.namespace == 'default':  # Focus on our apps
                pod_info = {
                    'name': pod.metadata.name,
                    'namespace': pod.metadata.namespace,
                    'status': pod.status.phase,
                    'node': pod.spec.node_name,
                    'app': pod.metadata.labels.get('app', 'unknown') if pod.metadata.labels else 'unknown'
                }
                cluster_info['pods'].append(pod_info)
        
        # Get services
        services = v1.list_service_for_all_namespaces()
        for svc in services.items:
            if svc.metadata.namespace == 'default':
                svc_info = {
                    'name': svc.metadata.name,
                    'type': svc.spec.type,
                    'cluster_ip': svc.spec.cluster_ip,
                    'ports': [f"{port.port}:{port.target_port}" for port in svc.spec.ports] if svc.spec.ports else []
                }
                cluster_info['services'].append(svc_info)
        
        return cluster_info
        
    except Exception as e:
        print(f"❌ Error getting cluster info: {e}")
        return None

# Get and display cluster information
print("🔍 Gathering cluster information...")
cluster_info = get_cluster_info()

if cluster_info:
    print("\n🏗️  Cluster Nodes:")
    nodes_df = pd.DataFrame(cluster_info['nodes'])
    print(nodes_df.to_string(index=False))
    
    print("\n🚀 Running Pods:")
    pods_df = pd.DataFrame(cluster_info['pods'])
    if not pods_df.empty:
        print(pods_df[['name', 'app', 'status', 'node']].to_string(index=False))
    
    print("\n🔗 Services:")
    services_df = pd.DataFrame(cluster_info['services'])
    if not services_df.empty:
        print(services_df[['name', 'type', 'cluster_ip']].to_string(index=False))
else:
    print("⚠️  Could not retrieve cluster information")

## 5. Vector Database Demo - Document Storage and Search

In [ ]:
# Create sample documents table with vector embeddings
def setup_vector_demo():
    """Set up vector database demo with sample data"""
    conn = get_db_connection()
    if not conn:
        return False
    
    try:
        with conn.cursor() as cur:
            # Create documents table with vector column
            cur.execute("""
                DROP TABLE IF EXISTS demo_documents;
                CREATE TABLE demo_documents (
                    id SERIAL PRIMARY KEY,
                    title TEXT NOT NULL,
                    content TEXT NOT NULL,
                    category TEXT,
                    embedding vector(384),  -- Using 384-dimension vectors (sentence-transformers size)
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)
            
            # Sample documents about our cluster
            sample_docs = [
                {
                    'title': 'Kubernetes Cluster Setup',
                    'content': 'Our three-node Kubernetes cluster consists of a tower control plane, AGX worker with GPU acceleration, and Nano edge device for lightweight workloads.',
                    'category': 'infrastructure'
                },
                {
                    'title': 'PostgreSQL with pgvector',
                    'content': 'PostgreSQL database enhanced with pgvector extension enables efficient vector similarity search for AI and machine learning applications.',
                    'category': 'database'
                },
                {
                    'title': 'GPU-Accelerated Computing',
                    'content': 'Both AGX and Nano nodes feature NVIDIA GPUs with CUDA support, enabling parallel processing for deep learning and AI workloads.',
                    'category': 'compute'
                },
                {
                    'title': 'FastAPI Microservices',
                    'content': 'Lightweight FastAPI applications deployed across worker nodes provide REST endpoints for machine learning inference and data processing.',
                    'category': 'application'
                },
                {
                    'title': 'Jupyter Lab Environment',
                    'content': 'Interactive Jupyter Lab notebook environment for data science, machine learning experimentation, and cluster monitoring.',
                    'category': 'development'
                },
                {
                    'title': 'Network Architecture',
                    'content': 'Multi-subnet network design with dedicated cluster communication, internet access, and external service exposure.',
                    'category': 'networking'
                }
            ]
            
            # Generate simple embeddings (in real scenario, use sentence-transformers)
            for doc in sample_docs:
                # Simple embedding based on word frequency (demo purposes)
                words = doc['content'].lower().split()
                embedding = np.random.normal(0, 1, 384).tolist()  # Random for demo
                
                cur.execute("""
                    INSERT INTO demo_documents (title, content, category, embedding)
                    VALUES (%s, %s, %s, %s)
                """, (doc['title'], doc['content'], doc['category'], embedding))
            
            # Create vector index for performance
            cur.execute("""
                CREATE INDEX IF NOT EXISTS demo_documents_embedding_idx 
                ON demo_documents USING ivfflat (embedding vector_cosine_ops)
                WITH (lists = 100);
            """)
            
            conn.commit()
            print("✅ Vector database demo setup complete!")
            
            # Show sample data
            cur.execute("SELECT title, category, created_at FROM demo_documents ORDER BY id;")
            results = cur.fetchall()
            
            df = pd.DataFrame(results, columns=['Title', 'Category', 'Created'])
            print("\n📚 Sample Documents:")
            print(df.to_string(index=False))
            
            return True
            
    except Exception as e:
        print(f"❌ Error setting up vector demo: {e}")
        return False
    finally:
        conn.close()

setup_vector_demo()

In [ ]:
# Vector similarity search demo
def vector_search_demo(query_text, limit=3):
    """Demonstrate vector similarity search"""
    conn = get_db_connection()
    if not conn:
        return
    
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            # Generate query embedding (simplified for demo)
            query_embedding = np.random.normal(0, 1, 384).tolist()
            
            # Perform vector similarity search
            cur.execute("""
                SELECT 
                    title,
                    content,
                    category,
                    embedding <=> %s::vector AS distance
                FROM demo_documents
                ORDER BY embedding <=> %s::vector
                LIMIT %s;
            """, (query_embedding, query_embedding, limit))
            
            results = cur.fetchall()
            
            print(f"🔍 Vector Search Results for: '{query_text}'")
            print("=" * 60)
            
            for i, result in enumerate(results, 1):
                print(f"\n{i}. {result['title']} (Distance: {result['distance']:.4f})")
                print(f"   Category: {result['category']}")
                print(f"   Content: {result['content'][:100]}...")
                
    except Exception as e:
        print(f"❌ Vector search error: {e}")
    finally:
        conn.close()

# Demo searches
search_queries = [
    "GPU computing and machine learning",
    "database vector search",
    "network cluster architecture"
]

for query in search_queries:
    vector_search_demo(query)
    print("\n" + "="*80 + "\n")

## 6. Service Health Monitoring

In [ ]:
def check_service_health():
    """Check health of cluster services"""
    health_status = []
    
    # Check FastAPI Nano service
    try:
        response = requests.get('http://fastapi-nano-service:8000/health', timeout=5)
        if response.status_code == 200:
            health_status.append({'Service': 'FastAPI Nano', 'Status': '✅ Healthy', 'Response Time': f"{response.elapsed.total_seconds():.3f}s"})
        else:
            health_status.append({'Service': 'FastAPI Nano', 'Status': f'⚠️  Status {response.status_code}', 'Response Time': 'N/A'})
    except Exception as e:
        health_status.append({'Service': 'FastAPI Nano', 'Status': f'❌ {str(e)[:50]}', 'Response Time': 'N/A'})
    
    # Check PostgreSQL
    try:
        conn = get_db_connection()
        if conn:
            with conn.cursor() as cur:
                start_time = datetime.now()
                cur.execute('SELECT 1')
                end_time = datetime.now()
                response_time = (end_time - start_time).total_seconds()
                health_status.append({'Service': 'PostgreSQL', 'Status': '✅ Healthy', 'Response Time': f"{response_time:.3f}s"})
            conn.close()
        else:
            health_status.append({'Service': 'PostgreSQL', 'Status': '❌ Connection Failed', 'Response Time': 'N/A'})
    except Exception as e:
        health_status.append({'Service': 'PostgreSQL', 'Status': f'❌ {str(e)[:50]}', 'Response Time': 'N/A'})
    
    # Check pgAdmin (if accessible)
    try:
        response = requests.get('http://pgadmin-service:80', timeout=5, allow_redirects=False)
        if response.status_code in [200, 302]:  # 302 is redirect to login
            health_status.append({'Service': 'pgAdmin', 'Status': '✅ Healthy', 'Response Time': f"{response.elapsed.total_seconds():.3f}s"})
        else:
            health_status.append({'Service': 'pgAdmin', 'Status': f'⚠️  Status {response.status_code}', 'Response Time': 'N/A'})
    except Exception as e:
        health_status.append({'Service': 'pgAdmin', 'Status': f'❌ {str(e)[:50]}', 'Response Time': 'N/A'})
    
    # Display results
    health_df = pd.DataFrame(health_status)
    print("🏥 Service Health Check")
    print(health_df.to_string(index=False))
    
    return health_df

health_results = check_service_health()

## 7. Cluster Resource Visualization

In [ ]:
# Visualize cluster resources
if cluster_info and cluster_info['nodes']:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Kubernetes Cluster Resource Overview', fontsize=16, fontweight='bold')
    
    nodes_df = pd.DataFrame(cluster_info['nodes'])
    
    # Node status pie chart
    status_counts = nodes_df['status'].value_counts()
    axes[0, 0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Node Status Distribution')
    
    # Architecture distribution
    arch_counts = nodes_df['arch'].value_counts()
    axes[0, 1].bar(arch_counts.index, arch_counts.values, color='skyblue')
    axes[0, 1].set_title('Node Architecture')
    axes[0, 1].set_ylabel('Count')
    
    # GPU availability
    gpu_data = nodes_df[['name', 'gpu']].copy()
    gpu_data['gpu'] = gpu_data['gpu'].astype(int)
    axes[1, 0].bar(gpu_data['name'], gpu_data['gpu'], color='green')
    axes[1, 0].set_title('GPU Resources per Node')
    axes[1, 0].set_ylabel('GPU Count')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Service distribution
    if cluster_info['services']:
        services_df = pd.DataFrame(cluster_info['services'])
        service_types = services_df['type'].value_counts()
        axes[1, 1].pie(service_types.values, labels=service_types.index, autopct='%1.1f%%')
        axes[1, 1].set_title('Service Type Distribution')
    else:
        axes[1, 1].text(0.5, 0.5, 'No services data', ha='center', va='center')
        axes[1, 1].set_title('Service Type Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Node details table
    print("\n📊 Detailed Node Information:")
    display_columns = ['name', 'status', 'arch', 'cpu', 'memory', 'gpu']
    print(nodes_df[display_columns].to_string(index=False))
else:
    print("⚠️  No cluster information available for visualization")

## 8. Database Performance Analysis

In [ ]:
def analyze_database_performance():
    """Analyze database performance and vector operations"""
    conn = get_db_connection()
    if not conn:
        return
    
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            # Database statistics
            cur.execute("""
                SELECT 
                    schemaname,
                    tablename,
                    n_tup_ins as inserts,
                    n_tup_upd as updates,
                    n_tup_del as deletes,
                    seq_scan as sequential_scans,
                    idx_scan as index_scans
                FROM pg_stat_user_tables
                WHERE tablename = 'demo_documents';
            """)
            
            stats = cur.fetchone()
            
            if stats:
                print("📈 Database Performance Metrics:")
                stats_df = pd.DataFrame([dict(stats)])
                print(stats_df.to_string(index=False))
            
            # Vector index information
            cur.execute("""
                SELECT 
                    indexname,
                    indexdef
                FROM pg_indexes 
                WHERE tablename = 'demo_documents' AND indexname LIKE '%embedding%';
            """)
            
            indexes = cur.fetchall()
            
            if indexes:
                print("\n🔍 Vector Indexes:")
                for idx in indexes:
                    print(f"  • {idx['indexname']}")
            
            # Extension information
            cur.execute("""
                SELECT 
                    extname as extension,
                    extversion as version,
                    extrelocatable as relocatable
                FROM pg_extension 
                WHERE extname IN ('vector', 'plpgsql');
            """)
            
            extensions = cur.fetchall()
            
            if extensions:
                print("\n🔧 Database Extensions:")
                ext_df = pd.DataFrame([dict(ext) for ext in extensions])
                print(ext_df.to_string(index=False))
                
    except Exception as e:
        print(f"❌ Error analyzing database performance: {e}")
    finally:
        conn.close()

analyze_database_performance()

## 9. Cluster Summary and Next Steps

In [ ]:
print("🎉 Kubernetes Cluster Demo Summary")
print("=" * 50)
print()
print("✅ Successfully demonstrated:")
print("  • Multi-node Kubernetes cluster operation")
print("  • PostgreSQL with pgvector extension")
print("  • Vector similarity search capabilities")
print("  • Service health monitoring")
print("  • Cluster resource visualization")
print("  • GPU resource detection")
print()
print("🔗 External Access URLs:")
print("  • Jupyter Lab: http://192.168.10.1:30888 (token: jupyter-k8s-demo)")
print("  • pgAdmin: http://192.168.10.1:30080 (pgadmin@pgadmin.org/pgadmin)")
print("  • PostgreSQL: 192.168.10.1:30432 (postgres/postgres)")
print()
print("🚀 Next Steps:")
print("  1. Deploy machine learning models on GPU nodes")
print("  2. Implement distributed training workflows")
print("  3. Set up monitoring with Prometheus + Grafana")
print("  4. Add CI/CD pipelines with GitOps")
print("  5. Implement auto-scaling policies")
print()
print("📚 Additional Features Available:")
print("  • NFS shared storage across all nodes")
print("  • Private container registry")
print("  • GPU scheduling and resource management")
print("  • Multi-architecture support (ARM64/AMD64)")
print()
print(f"⏰ Demo completed at: {datetime.now()}")

## 10. Nano Agent Setup Process

The nano node is currently in **NotReady** status. The nano agent has a dedicated setup script that performs comprehensive initialization including container build, k3s agent setup, and service deployment.

### Complete Agent Setup Process

The `setup_fastapi_nano.sh` script performs these operations:

#### 🏗️ Container Build Process
1. **Build from Dockerfile + devcontainer.json**
   - Uses nano-specific dockerfile.nano.req
   - Includes devcontainer.json configuration
   - **Jupyter Lab is pre-installed** in the base image
   - Builds FastAPI nano application container

#### ⚙️ Kubernetes Agent Setup
2. **Install k3s Agent**
   - Download and install k3s agent
   - Configure with tower server token
   - Set up secure certificates
   - Configure insecure registry access

3. **Join Cluster**
   - Connect to tower control plane (192.168.5.1:6443)
   - Register as worker node
   - Establish cluster networking

#### 🚀 Application Deployment
4. **Deploy FastAPI Application**
   - Execute fastapi_app.py
   - Configure nano-specific settings
   - Set up health checks (fastapi_healthcheck.py)
   - Initialize database if needed (init_db.sql)

5. **Start Services**
   - **FastAPI**: Port 8000 (NodePort 30002)
   - **Jupyter Lab**: Via start-jupyter.sh
   - Both services accessible via cluster network

### Available Scripts on Nano Device
```bash
# Located in: ~/containers/kubernetes/agent/nano/src/
backup_home.sh              # Backup nano home directory
fastapi_app.py              # Main FastAPI application
fastapi_healthcheck.py      # Health check endpoint
init_db.sql                 # Database initialization
main.py                     # Application entry point
restart_fastapi_nano.sh     # Restart FastAPI service
setup_fastapi_nano.sh       # MAIN SETUP SCRIPT
start-jupyter.sh            # Start Jupyter Lab service
```

### Execution Steps (You're Already on Nano!)
```bash
# Current location: sanjay@nano:~/containers/kubernetes/agent/nano/src$

# Execute the main setup script
bash setup_fastapi_nano.sh

# Optional: Run with debug for detailed output
DEBUG=1 bash setup_fastapi_nano.sh
```

### What the Script Will Accomplish
- ✅ **Container Build**: FastAPI nano image with Jupyter pre-installed
- ✅ **k3s Agent**: Properly configured and joined to cluster
- ✅ **FastAPI Service**: Running fastapi_app.py on port 8000
- ✅ **Jupyter Service**: Available via start-jupyter.sh
- ✅ **Health Monitoring**: fastapi_healthcheck.py operational
- ✅ **Database Setup**: init_db.sql if database required
- ✅ **Service Restart**: restart_fastapi_nano.sh for maintenance

### Expected Final State
```
nano node: NotReady → Ready
fastapi-nano pod: Pending → Running (on nano node only)
FastAPI: http://192.168.10.1:30002
Jupyter: Available within nano container
Health Check: /health endpoint operational
```

In [ ]:
# Nano device is ready for setup script execution
print("🎯 Nano Agent Setup Ready for Execution")
print("=" * 50)

# Check current cluster status from tower perspective
import subprocess
import json

try:
    # Get node status
    result = subprocess.run(['kubectl', 'get', 'nodes', '-o', 'json'], 
                          capture_output=True, text=True, timeout=10)
    
    if result.returncode == 0:
        nodes_data = json.loads(result.stdout)
        
        print("\n📊 Current Cluster Status (from Tower):")
        for node in nodes_data['items']:
            name = node['metadata']['name']
            
            # Check ready condition
            ready_status = "Unknown"
            for condition in node['status']['conditions']:
                if condition['type'] == 'Ready':
                    ready_status = "Ready" if condition['status'] == 'True' else "NotReady"
                    break
            
            status_emoji = "✅" if ready_status == "Ready" else "❌"
            print(f"  {status_emoji} {name}: {ready_status}")
            
            if name == 'nano':
                print(f"      └── 🎯 User is positioned on nano device!")
                print(f"      └── 📁 Location: ~/containers/kubernetes/agent/nano/src/")
                print(f"      └── 🔧 Ready to run: setup_fastapi_nano.sh")
    
    # Check FastAPI nano pod status
    result = subprocess.run(['kubectl', 'get', 'pods', '-l', 'app=fastapi-nano', '-o', 'json'], 
                          capture_output=True, text=True, timeout=10)
    
    if result.returncode == 0:
        pods_data = json.loads(result.stdout)
        
        print(f"\n🚀 FastAPI Nano Pod Status:")
        if pods_data['items']:
            for pod in pods_data['items']:
                name = pod['metadata']['name']
                status = pod['status']['phase']
                
                status_emoji = "✅" if status == "Running" else "⏳" if status == "Pending" else "❌"
                print(f"  {status_emoji} {name}: {status}")
                
                if status == "Pending":
                    print(f"      └── ⏳ Waiting for nano agent setup completion")
        else:
            print("  ℹ️  No FastAPI nano pods found")

    print(f"\n🛠️ Available Setup Scripts on Nano:")
    nano_scripts = [
        "setup_fastapi_nano.sh       # 🎯 MAIN SETUP SCRIPT",
        "fastapi_app.py              # FastAPI application",
        "fastapi_healthcheck.py      # Health monitoring",
        "start-jupyter.sh            # Jupyter Lab startup",
        "restart_fastapi_nano.sh     # Service restart",
        "backup_home.sh              # Home backup utility",
        "init_db.sql                 # Database initialization",
        "main.py                     # Application entry point"
    ]
    
    for script in nano_scripts:
        print(f"  📜 {script}")

    print(f"\n⚡ Ready to Execute:")
    print(f"  💻 Current position: sanjay@nano:~/containers/kubernetes/agent/nano/src$")
    print(f"  🚀 Command: bash setup_fastapi_nano.sh")
    print(f"  🔍 Debug mode: DEBUG=1 bash setup_fastapi_nano.sh")
    print(f"")
    print(f"🎯 This will:")
    print(f"  • Build nano container with Jupyter + FastAPI")
    print(f"  • Install and configure k3s agent")
    print(f"  • Join nano node to cluster")
    print(f"  • Deploy and start all services")
    print(f"  • Make nano node Ready and pod Running")

except subprocess.TimeoutExpired:
    print("❌ Kubectl command timed out")
except subprocess.CalledProcessError as e:
    print(f"❌ Kubectl command failed: {e}")
except json.JSONDecodeError:
    print("❌ Failed to parse kubectl output")
except Exception as e:
    print(f"❌ Error checking cluster status: {e}")

print(f"\n✨ You're perfectly positioned to run the setup!")
print(f"   Execute: bash setup_fastapi_nano.sh")

### Jupyter Lab Architecture in Our Cluster

Our cluster has **two distinct Jupyter Lab instances**:

#### 🏗️ Tower Jupyter Lab (Current Instance)
- **Location**: Running on tower node (control plane)
- **Purpose**: Cluster monitoring, admin tasks, this demo notebook
- **Access**: http://192.168.10.1:30888
- **Image**: jupyter/tensorflow-notebook:latest
- **Status**: Currently running (where this notebook executes)

#### 🤖 Nano Jupyter Lab (Agent Container)
- **Location**: Will run inside FastAPI nano container
- **Purpose**: Edge computing, nano-specific development
- **Access**: Internal to nano container (part of agent setup)
- **Image**: Built with dockerfile.nano.req + devcontainer.json
- **Status**: Will be available after nano agent setup completes

Both instances serve different purposes:
- **Tower**: Cluster-wide monitoring and administration
- **Nano**: Edge-specific development and testing